# Files tree

```
./datasets
|
+---------MedicalDecathlon
|                   |
|                   +-----training
|                   |         |
|                   |         |
|                   |         +--- archives with the cts in nii.gz format
|                   |
|                   |
|                   |
|                   +-----validation
|                             |
|                             |
|                             +--- archives with the cts in nii.gz format
|
|
|
+---------MedicalDecathlonTensors
|                   |
|                   +-----training
|                   |         |
|                   |         |
|                   |         +--- files with the cts as tensors
|                   |
|                   |
|                   |
|                   +-----validation
|                             |
|                             |
|                             +---  files with the cts as tensors
|
|
|
|
+---------MedicalDecathlonJustLungs
|                   |
|                   +-----training
|                   |         |
|                   |         |
|                   |         +--- files with the cts as tensors
|                   |
|                   |
|                   |
|                   +-----validation
|                             |
|                             |
|                             +---  files with the cts as tensors
|
|
|
+---------MedicalDecathlonJustTumours
                    |
                    +-----training
                    |         |
                    |         |
                    |         +--- files with the cts as tensors
                    |
                    |
                    |
                    +-----validation
                              |
                              |
                              +---  files with the cts as tensors


```


### IMPORTS

In [14]:
import os
import warnings
warnings.filterwarnings("ignore") # remove some scikit-image warnings


from monai.apps import DecathlonDataset
from monai.data import DataLoader
from monai.transforms import (
    LoadImageD,
    EnsureChannelFirstD,
    Compose,
    OrientationD,
    OrientationD,
)

import torch
import numpy as np
import random
import sys
from tqdm import tqdm
import pickle as pkl

### HYPERPARAMS ###


In [15]:
crt_dir = os.getcwd()
datasets_path = f'{crt_dir}/datasets/MedicalDecathlon/'

DOWNLOAD_FLAG = False
BATCH_SIZE = 1

KEYS = ["image", "label"]

### PREPROCESSING TRANSFORMS

In [16]:
base_transform = Compose([
    LoadImageD(keys=KEYS),
    EnsureChannelFirstD(keys=KEYS),
    OrientationD(keys=KEYS, axcodes='RAS'),
])

### MAIN ###

In [17]:
# We make sure to eliminate uncertainty in the data loading, by setting all the random seeds

# Set random seeds
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [18]:
# Memoize stacks in between, where we find the lungs
# Based on the radiologist, some CTs start from kidney area up to the cervical backbone
patient_useful_slices = {
    0 : {'start': 60, 'stop': 557},
    1 : {'start': 20 , 'stop': 243},
    2 : {'start': 0, 'stop': 213},
    3 : {'start': 48, 'stop': 437},
    4 : {'start': 52, 'stop': 258},
    5 : {'start': 59, 'stop': 244},
    6 : {'start': 9, 'stop': 122},
    7 : {'start': 44, 'stop': 260},
    8 : {'start': 3, 'stop': 237},
    9 : {'start': 330 , 'stop': 623},
    10 : {'start': 56, 'stop': 280},
    11 : {'start': 26, 'stop': 292},
    12 : {'start': 0, 'stop': 108},
    13 : {'start': 10, 'stop': 227},
    14 : {'start': 25, 'stop': 231},
    15 : {'start': 35, 'stop': 280},
    16 : {'start': 37, 'stop': 262},
    17 : {'start': 29, 'stop': 266},
    18 : {'start': 48, 'stop': 261},
    19 : {'start': 65, 'stop': 275},
    20 : {'start': 145, 'stop': 520},
    21 : {'start': 0, 'stop': 397},
    22 : {'start': 9, 'stop': 225},
    23 : {'start': 6, 'stop': 227},
    24 : {'start': 15, 'stop': 239},
    25 : {'start': 40, 'stop': 149},
    26 : {'start': 30, 'stop': 231},
    27 : {'start': 49, 'stop': 295},
    28 : {'start': 33, 'stop': 217},
    29 : {'start': 50, 'stop': 311},
    30 : {'start': 0, 'stop': 213},
    31 : {'start': 22, 'stop': 248},
    32 : {'start': 88, 'stop': 493},
    33 : {'start': 116, 'stop': 439},
    34 : {'start': 40, 'stop': 300},
    35 : {'start': 0, 'stop': 227},
    36 : {'start': 0, 'stop': 198},
    37 : {'start': 0, 'stop': 120},
    38 : {'start': 0, 'stop': 203},
    39 : {'start': 44, 'stop': 228},
    40 : {'start': 0, 'stop': 229},
    41 : {'start': 35, 'stop': 250},
    42 : {'start': 48, 'stop': 243},
    43 : {'start': 5, 'stop': 230},
    44 : {'start': 88, 'stop': 314},
    45 : {'start': 7, 'stop': 220},
    46 : {'start': 11, 'stop': 232},
    47 : {'start': 21, 'stop': 229},
    48 : {'start': 4, 'stop': 255},
    49 : {'start': 2, 'stop': 122},
    50 : {'start': 14, 'stop': 119}
}

In [19]:
# Memoize stacks where the tumor is present
# Computed in MSD EDA notebook
patient_tumorous_slices = {
    0: {'start': 326, 'stop': 411},
    1: {'start': 171, 'stop': 227},
    2: {'start': 147, 'stop': 193},
    3: {'start': 303, 'stop': 335},
    4: {'start': 141, 'stop': 167},
    5: {'start': 131, 'stop': 188},
    6: {'start': 71, 'stop': 95},
    7: {'start': 164, 'stop': 192},
    8: {'start': 145, 'stop': 180},
    9: {'start': 513, 'stop': 549},
    10: {'start': 197, 'stop': 272},
    11: {'start': 223, 'stop': 258},
    12: {'start': 59, 'stop': 83},
    13: {'start': 59, 'stop': 99},
    14: {'start': 178, 'stop': 207},
    15: {'start': 141, 'stop': 172},
    16: {'start': 127, 'stop': 177},
    17: {'start': 192, 'stop': 221},
    18: {'start': 209, 'stop': 239},
    19: {'start': 152, 'stop': 194},
    20: {'start': 439, 'stop': 494},
    21: {'start': 291, 'stop': 341},
    22: {'start': 136, 'stop': 180},
    23: {'start': 137, 'stop': 169},
    24: {'start': 84, 'stop': 112},
    25: {'start': 82, 'stop': 109},
    26: {'start': 79, 'stop': 126},
    27: {'start': 208, 'stop': 293},
    28: {'start': 137, 'stop': 173},
    29: {'start': 120, 'stop': 149},
    30: {'start': 130, 'stop': 173},
    31: {'start': 97, 'stop': 123},
    32: {'start': 356, 'stop': 415},
    33: {'start': 203, 'stop': 287},
    34: {'start': 200, 'stop': 228},
    35: {'start': 148, 'stop': 184},
    36: {'start': 79, 'stop': 105},
    37: {'start': 66, 'stop': 100},
    38: {'start': 117, 'stop': 143},
    39: {'start': 111, 'stop': 181},
    40: {'start': 140, 'stop': 165},
    41: {'start': 80, 'stop': 109},
    42: {'start': 79, 'stop': 172},
    43: {'start': 153, 'stop': 204},
    44: {'start': 147, 'stop': 188},
    45: {'start': 113, 'stop': 142},
    46: {'start': 83, 'stop': 113},
    47: {'start': 140, 'stop': 234},
    48: {'start': 141, 'stop': 170},
    49: {'start': 99, 'stop': 120},
    50: {'start': 68, 'stop': 92}
}

In [20]:
### TRAINING DATA ###
train_name = 'training' # From Monai: ['training', 'validation', 'test']
train_dataset = DecathlonDataset(root_dir = f'{datasets_path}{train_name}/',
                        task = "Task06_Lung", section = train_name,
                        transform = base_transform, download = DOWNLOAD_FLAG)

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = False) #, num_workers = num_workers)

Loading dataset: 100%|██████████| 51/51 [03:24<00:00,  4.01s/it]


In [35]:
def save_dataset(dataset, dataset_name, split_name, take_slices_flag : bool = False, slices : {int : {str : int}} = None):
    '''
    Save the dataset to the disk per patient as two files: image and mask'''
    i = 0

    if not os.path.exists(f'{crt_dir}/datasets/{dataset_name}/'):
        os.makedirs(f'{crt_dir}/datasets/{dataset_name}/')
        os.makedirs(f'{crt_dir}/datasets/{dataset_name}/{split_name}/images/')
        os.makedirs(f'{crt_dir}/datasets/{dataset_name}/{split_name}/labels/')


    if not os.path.exists(f'{crt_dir}/datasets/{dataset_name}/{split_name}/'):
        os.makedirs(f'{crt_dir}/datasets/{dataset_name}/{split_name}/')
        os.makedirs(f'{crt_dir}/datasets/{dataset_name}/{split_name}/images/')
        os.makedirs(f'{crt_dir}/datasets/{dataset_name}/{split_name}/labels/')

    for data in tqdm(dataset):
        img, label = data['image'], data['label']
        
        
        img = img.cpu()
        label = label.cpu()


        if take_slices_flag:
            img = img[:, :, :, slices[i]['start'] : slices[i]['stop']+1]
            label = label[:, :, :, slices[i]['start'] : slices[i]['stop']+1]

        print(f'Patient {i} has {img.shape[-1]} slices')

        filename = f'{crt_dir}/datasets/{dataset_name}/{split_name}/images/patient_{i}.pt'
        maskname = f'{crt_dir}/datasets/{dataset_name}/{split_name}/labels/patient_{i}.pt'

        torch.save(img, filename)
        torch.save(label, maskname)

        i += 1

    print(f'Saved {i} images and masks for {split_name} split')

In [31]:
save_dataset(train_dataset, 'MedicalDecathlonTensors', train_name)

100%|██████████| 51/51 [00:00<00:00, 6763.08it/s]

torch.Size([1, 512, 512, 589])
Patient 0 has 589 slices
torch.Size([1, 512, 512, 267])
Patient 1 has 267 slices
torch.Size([1, 512, 512, 227])
Patient 2 has 227 slices
torch.Size([1, 512, 512, 450])
Patient 3 has 450 slices
torch.Size([1, 512, 512, 277])
Patient 4 has 277 slices
torch.Size([1, 512, 512, 260])
Patient 5 has 260 slices
torch.Size([1, 512, 512, 125])
Patient 6 has 125 slices
torch.Size([1, 512, 512, 273])
Patient 7 has 273 slices
torch.Size([1, 512, 512, 251])
Patient 8 has 251 slices
torch.Size([1, 512, 512, 636])
Patient 9 has 636 slices
torch.Size([1, 512, 512, 301])
Patient 10 has 301 slices
torch.Size([1, 512, 512, 304])
Patient 11 has 304 slices
torch.Size([1, 512, 512, 115])
Patient 12 has 115 slices
torch.Size([1, 512, 512, 241])
Patient 13 has 241 slices
torch.Size([1, 512, 512, 249])
Patient 14 has 249 slices
torch.Size([1, 512, 512, 289])
Patient 15 has 289 slices
torch.Size([1, 512, 512, 282])
Patient 16 has 282 slices
torch.Size([1, 512, 512, 291])
Patient 17

In [36]:
save_dataset(train_dataset, 'MedicalDecathlonJustLungs', train_name, take_slices_flag = True, slices = patient_useful_slices)

  0%|          | 0/51 [00:00<?, ?it/s]

Patient 0 has 498 slices


  2%|▏         | 1/51 [00:04<03:20,  4.02s/it]

Patient 1 has 224 slices


  4%|▍         | 2/51 [00:05<02:13,  2.73s/it]

...

Patient 49 has 121 slices


 98%|█████████▊| 50/51 [01:34<00:01,  1.43s/it]

Patient 50 has 106 slices


100%|██████████| 51/51 [01:34<00:00,  1.86s/it]

Saved 51 images and masks for training split


In [37]:
save_dataset(train_dataset, 'MedicalDecathlonJustTumors', train_name, take_slices_flag = True, slices = patient_tumorous_slices)

  0%|          | 0/51 [00:00<?, ?it/s]

Patient 0 has 86 slices


  2%|▏         | 1/51 [00:03<03:17,  3.96s/it]

Patient 1 has 57 slices


  4%|▍         | 2/51 [00:05<02:12,  2.71s/it]

...

Patient 49 has 22 slices


 98%|█████████▊| 50/51 [01:51<00:01,  1.96s/it]

Patient 50 has 25 slices


100%|██████████| 51/51 [01:52<00:00,  2.21s/it]

Saved 51 images and masks for training split


In [6]:
### VALIDATION DATA ###
val_name = 'validation' # From Monai: ['training', 'validation', 'test']
val_dataset = DecathlonDataset(root_dir = f'{datasets_path}{val_name}/',
                        task = "Task06_Lung", section = val_name,
                        transform = base_transform, download = DOWNLOAD_FLAG)

val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False) #, num_workers = num_workers)


Loading dataset: 100%|██████████| 12/12 [00:46<00:00,  3.88s/it]


In [10]:
val_dataset[0]

{'image': metatensor([[[[ -836.,  -894.,  -940.,  ...,  -908.,  -908.,  -886.],
           [ -839.,  -871.,  -923.,  ...,  -928.,  -955.,  -925.],
           [ -928.,  -901.,  -914.,  ...,  -914.,  -930.,  -929.],
           ...,
           [ -988.,  -984.,  -985.,  ...,  -990., -1001.,  -965.],
           [ -971.,  -975., -1014.,  ...,  -998., -1005.,  -986.],
           [ -996., -1002., -1024.,  ...,  -987., -1011., -1015.]],
 
          [[ -902.,  -909.,  -901.,  ...,  -910.,  -879.,  -899.],
           [ -838.,  -899.,  -942.,  ...,  -890.,  -915.,  -920.],
           [ -864.,  -901.,  -923.,  ...,  -891.,  -949.,  -923.],
           ...,
           [ -963.,  -973.,  -981.,  ..., -1002.,  -997.,  -993.],
           [ -990.,  -994., -1024.,  ...,  -992.,  -999., -1003.],
           [-1006., -1013., -1021.,  ...,  -975.,  -997., -1023.]],
 
          [[ -942.,  -898.,  -882.,  ...,  -940.,  -901.,  -918.],
           [ -884.,  -901.,  -889.,  ...,  -882.,  -888.,  -908.],
           

In [12]:
save_dataset(val_dataset, 'MedicalDecathlonJustTumors', val_name)

  0%|          | 0/12 [00:00<?, ?it/s]

Patient 0 has 369 slices


 25%|██▌       | 3/12 [00:02<00:05,  1.53it/s]

Patient 1 has 255 slices
Patient 2 has 288 slices


 42%|████▏     | 5/12 [00:03<00:03,  1.82it/s]

...

Patient 7 has 231 slices
Patient 8 has 129 slices


 92%|█████████▏| 11/12 [00:06<00:00,  1.51it/s]

Patient 9 has 240 slices
Patient 10 has 493 slices


100%|██████████| 12/12 [00:07<00:00,  1.67it/s]

Patient 11 has 264 slices
Saved 12 images and masks for validation split
